In [22]:
import youtube_dl
from pydub import AudioSegment
import pathlib
import os
import csv
import numpy as np
import librosa


def timestamps(total_time, split):
    iters = int(total_time/split)
    for i in range(1, iters + 1):
        yield (i-1)*split, i * split, i

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()


        
def get_audio_from_url(url, title, split=2500):

    ydl_opts = {
        'outtmpl': f'audio/FULL_INTERVIEWS/{title}.mp3',
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'prefer_ffmpeg': True,

    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([f"{url}"])
        
    sound = AudioSegment.from_file(f"audio/FULL_INTERVIEWS/{title}.mp3")
    sound.export(f"audio/FULL_INTERVIEWS/{title}.wav", format="wav")

    
    pathlib.Path(f'audio/{title}').mkdir(parents=True, exist_ok=True)

    newAudio = AudioSegment.from_wav(f"audio/FULL_INTERVIEWS/{title}.wav")

    for start, end, val in timestamps(len(newAudio), split):
        cutAudio = newAudio[start:end]
        cutAudio.export(f'audio/{title}/{title}_{val-1}.wav', format='wav')   
    
        
        
    file = open(f'datasets/{title}.csv', 'w', newline='')

    with file:
        writer = csv.writer(file)
        writer.writerow(header)

    for filename in os.listdir(f'audio/{title}'):
        songname = f'audio/{title}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)

        rmse = librosa.feature.rms(y=y) #root mean squred ENERGY note. changed to rms form rmse
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)

        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

        for e in mfcc:
            to_append += f' {np.mean(e)}'

        to_append += f' {title}'

        file = open(f'datasets/{title}.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
    

In [24]:
get_audio_from_url('https://www.youtube.com/watch?v=eJFfoSQcC0A', 'dopamine')

[youtube] eJFfoSQcC0A: Downloading webpage
[download] Destination: audio\FULL_INTERVIEWS\dopamine.mp3
[download] 100% of 5.06MiB in 00:02                  
[ffmpeg] Post-process file audio\FULL_INTERVIEWS\dopamine.mp3 exists, skipping
